# Aircrew trainee scheduling with seniority constraints and preferences
[![tip8_aircrew_trainees_seniority.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/glebbelov/modeling-tips/tip8_aircrew_trainees_seniority.ipynb)](https://colab.ampl.com)

Description: Aircrew trainee scheduling with seniority constraints and preferences

Tags: trainee scheduling, aircrew scheduling, employee scheduling, seniority constraints, seniority ranking, amplpy

Notebook author: Gleb Belov <<gleb@ampl.com>>

References:
1. Kozanidis, G. (2017). Optimal assignment of aircrew trainees to simulator and classroom training sessions subject to seniority and preference restrictions. Journal of Air Transport Management 59, 143-154.
2. Gamache, M., Soumis, F., Villeneuve, D., Desrosiers, J., & Gélinas, É. (1998). The preferential bidding system at Air Canada. Transportation Science, 32(3), 246-255.
3. Achour, H., Gamache, M., Soumis, F., & Desaulniers, G. (2007). An exact solution approach for the preferential bidding system problem in the airline industry. Transportation Science, 41(3), 354-365.

In [1]:
# Install dependencies
%pip install -q amplpy pandas

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gurobi", "xpress"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Mathematical model

This notebook considers a realistic trainee scheduling model from [1] involving classroom capacities, language and (reverse) seniority constraints, and seniority-ranked preferences. The (compulsory) seniority constraints are modeled by Preferential Bidding System (PBS)-style secondary objective functions.

**Sets:**
- $I$ - set of crew trainees,
- $T$ - set of training sessions,
- $V_i$ - set of valid sessions of trainee $i\in I$,
- $Pos$ - set of crew positions:  captain (CP), first officer (FO), purser (PU), or flight attendant (FA),
- $Grp$ - set of trainee groups: all, cockpit (CK), cabin (CB).

**Parameters:**
- $P_i\in Pos$ - position of trainee $i\in I$,
- $S_i$ - seniority. The smaller this number, the more senior the corresponding crew member,
- $E_i$ - flight capability expiration. 0: end of the current month, 1: end of the next month, 2: end of the next to the following month,
- $L_i$ - spoken language. 0: the trainee speaks both of the alternative languages, 1: only the first language, 2: only the second language,
- $Pr_{it}$ - priority of trainee $i$ for valid session $t$. Priorities are non-negative integers. Higher priorities mean higher preference. Zero priority means that the trainee does not want to be assigned to this session. All positive priorities for the same trainee should be different, while zero priority can be expressed for multiple sessions,
- $MAX_p$ - capacity of each training session for trainees of position $p\in Pos$,
- $MAX^+_g$ - capacity of each training session for trainees of group $g\in Grp$.

The aim is to find an optimal assignment of trainees to training sessions which minimizes the number of training slots that remain empty, while also satisfying a list of hard and soft requirements. The satisfaction of the hard restrictions is unnegotiable, whereas the soft ones must be satisfied to the greatest possible extent. The **hard restrictions** that must be satisfied are the following:
- Capacity constraints for each training session,
- One of the two alternative languages must be selected for each session,
- Every crew member with expiration 0 must be assigned to one of his/her valid sessions,
- Reverse expiration and reverse seniority restrictions. Suppose that there exists a slot in a training session, which is only valid to a specific subset of crew members, none of which has expressed positive preference for being assigned to this session. Since the minimization of the total number of empty slots is the model's primary objective, one of these crew members must eventually be assigned to this session. The reverse expiration restrictions, considered first, dictate that this should be the crew member with the lowest expiration (his/her flight capability should expire sooner) among them. Assume now, that two or more trainees have the lowest expiration. The reverse seniority restrictions, considered next, dictate that this should be the least senior trainee among all the trainees with the lowest expiration.

Additionally, the following **soft restrictions** must be satisfied:
- The maximum possible preference satisfaction for each crew member must be attained, according to strict seniority. This means that no crew member's preference should be satisfied at the expense of the dissatisfaction of a more senior crew member's preference.
- The maximum possible preference satisfaction of each individual crew member must be attained, according to the list of priorities that he/she has expressed.

**Decision variables:**
- $x_{it}$ - binary decision variable, takes value $1$ *iff* trainee $i\in I$ is assigned to session $t\in V_t$,
- $y_i$ - binary decision variable that takes the value 1 *iff* trainee i remains unassigned, $i\in I$: $E_i = 1$ or $2$,
- $u_t$: binary decision variable that takes the value 1 if language 1 is selected for training session t, and 0 if language 2 is selected instead, $t\in T$,
- $w$: total number of crew trainees that remain unassigned.

The **optimization model**, formulated below in AMPL, follows [1], while differs as follows:
- The reverse expiration and seniority constraints are sublimated via Preferential Bidding System (PBS)-style secondary objective functions. It remains to be shown that minimizing the objective functions `ReverseSeniority[]` provides constraints (8), (9), (22-25), and (26--29) from [1], as well as resolves the 'pitfall' from Section 3.1.
- The lexicographic approach for preference ranking by seniority is implemented via multiple objectives submitted to the solver in a single model.
No variables are fixed while optimizing the different objectives.

## Test instance generation

Define class `Instance` holding a problem instance.

In [3]:
class Instance:
    def __init__(self):
        ## Data arrays / matrices for all trainees
        self.nT = 0
        self.P = []
        self.S = []
        self.L = []
        self.E = []
        self.V = []
        self.Pr = []
        ## Capacity constraints
        self.cap_kind = []
        self.MAX = []
        self.cap_plus_kind = []
        self.cap_plus_which = []
        self.MAXPlus = []

Define class `InstGenParams` with instance generation parameters according to [1].

In [4]:
import math

class InstGenParams:
    def __init__(self, N):
        self.N = N
        self.T = math.ceil(N/5)
        ## Ranges and probabilities
        self.position_kind = ["CP", "FO", "PU", "FA"]
        self.position_prob = [0.25, 0.25, 0.25, 0.25]
        self.language_prob = [0.5, 0.25, 0.25]
        self.expiration_prob = [0.5, 0.25, 0.25]
        self.N_sess_rng = [1, math.floor(self.T/2)]
        self.pref_0 = 0.5
        ## These are constant for all instances
        self.cap_kind = ["CP", "FO", "PU", "FA"]
        self.MAX = [4, 4, 4, 4]
        self.cap_plus_kind = ["All", "CK", "CB"]
        self.cap_plus_which = [[1, 2], [1, 2], [1, 3], [1, 3]]
        self.MAXPlus = [10, 6, 6]

Function to generate an instance.

In [5]:
import numpy as np
import pandas as pd

def GenInstance(genpar, rng):
    inst = Instance()
    inst.nT = genpar.T
    inst.S = np.arange(genpar.N)              # Seniority 0..N-1
    rng.shuffle(inst.S)
    for itr in range(0, genpar.N):
        inst.P.append(rng.choice(np.arange(1, len(genpar.position_prob)+1),  # 1-based
                                 p=genpar.position_prob))
        inst.L.append(rng.choice(len(genpar.language_prob), p=genpar.language_prob))
        inst.E.append(rng.choice(len(genpar.expiration_prob), p=genpar.expiration_prob))
        N_feas_sess = rng.integers(genpar.N_sess_rng[0], genpar.N_sess_rng[1]+1)
        V = rng.choice(np.arange(1, genpar.T+1),                   # 1-based
                       size=N_feas_sess, replace=False)
        inst.V.append( V )
        N_pr_0 = rng.binomial(N_feas_sess, genpar.pref_0)
        PrByV = np.concatenate( (np.zeros(N_pr_0), np.arange(1, N_feas_sess-N_pr_0+1)) )
        assert N_feas_sess == len(PrByV)
        rng.shuffle(PrByV)
        Pr = np.zeros(genpar.T)
        for k, t in enumerate(V):
            Pr[t-1] = PrByV[k]
        inst.Pr.append( Pr )
    # Capacities
    inst.cap_kind = genpar.cap_kind
    inst.MAX = genpar.MAX
    inst.cap_plus_kind = genpar.cap_plus_kind
    inst.cap_plus_which = genpar.cap_plus_which
    inst.MAXPlus = genpar.MAXPlus
    return inst

Generate instance.

In [6]:
rng = np.random.default_rng(122807528840384100672342137672332424406)

genpar = InstGenParams(200)             # With the Demo license, set up to 50
inst = GenInstance(genpar, rng)

## AMPL model

The below model is saved to file `airtrainee.mod`.

In [7]:
%%writefile airtrainee.mod

set I;              # Trainees
set T;              # Training sessions
set V{I} in T;      # Valid sessions for each trainee
set Pos;            # Positons
set PosPlus;        # Meta-positions (e.g, All, Cockpit, Cabin)
set MAXPlusWhich{Pos} in PosPlus;

param P{I} in Pos;  # Trainee's position
param S{I};         # Seniority (smaller value <=> higher seniority)
param L{I};         # Language (0 - both, 1 or 2 - one only)
param E{I};         # Expiration: 0 - this month, 1 - next month, 2 - in 2 months
param Pr{I, T};     # Priority: 0 - not wanted, larger value <=> higher preference
param MAX{Pos};     # Position capacity
param MAXPlus{PosPlus};  # Aggregated capacities (All, CK, CB)

var x{i in I, t in T} binary <= if t in V[i] then 1 else 0;
var y{i in I} binary <= if E[i]>0 then 1 else 0;      # Trainee i unassigned
var u{T} binary;    # 1 <=> language 1, 0 <=> language 2
var w >=0;          # Number of unassigned trainees

s.t. Assign_E0 {i in I: E[i]==0}:
  sum {t in V[i]} x[i, t] == 1;

s.t. Unassigned_E12 {i in I: E[i]>0}:
  sum {t in V[i]} x[i, t] + y[i] == 1;

s.t. Sum_Unassigned: w == sum {i in I: E[i]>0} y[i];

s.t. Language_1 {t in T}:
  u[t]<0.5 ==> sum {i in I: L[i]==1 and t in V[i]} x[i, t] <= 0;

s.t. Language_2 {t in T}:
  u[t]>=0.5 ==> sum {i in I: L[i]==2 and t in V[i]} x[i, t] <= 0;

s.t. Capacity {p in Pos, t in T}:
  sum {i in I: p==P[i] and t in V[i]} x[i, t] <= MAX[p];

s.t. Capacity_Meta {p in PosPlus, t in T}:
  sum {i in I: p in MAXPlusWhich[P[i]] and t in V[i]} x[i, t] <= MAXPlus[p];

suffix objpriority;
param S_range := max {i in I} S[i] - min {i in I} S[i];

# The primary objective
minimize Total_Unassigned: w suffix objpriority S_range + 4;
# let Total_Unassigned.objpriority := card(I) + 3;

# Trainee preferences, ranked by seniority
set SenLevels := setof {i in I} S[i];
param prefMax {i in I} := max {t in V[i]} Pr[i, t];
minimize PrefViolRanked {s in SenLevels}:
  sum {i in I, t in T: s==S[i] and t in V[i]}
      ((prefMax[i] - Pr[i, t]) / max(prefMax[i], 1)           # Normalize
       * x[i, t])
    suffix objpriority max {i in I} S[i] + 3 - s;

# Reverse seniority constraints,
# sublimated as post-processing phase objectives.
# Optimize for all trainees with E[i]==1,
# then for all with E[i]==2.
maximize ReverseSeniority {e in 1..2}:
  sum {i in I, t in T: E[i]==e and t in V[i] and Pr[i, t]==0}
    S[i] * x[i, t]
    suffix objpriority 3-e;

Overwriting airtrainee.mod


Function to pass an instance to the AMPL model.

In [8]:
def MakeAMPLInstance(inst):
    ampl = AMPL()
    ampl.read("airtrainee.mod")
    # Fill data
    ampl.set["I"] = np.arange(1, len(inst.P)+1)
    ampl.set["T"] = np.arange(1, inst.nT+1)
    for i, vi in enumerate(inst.V):
        ampl.set["V"][i+1] = vi
    ampl.set["Pos"] = np.arange(1, len(genpar.position_prob)+1)
    ampl.set["PosPlus"] = np.arange(1, len(genpar.MAXPlus)+1)
    for i, vi in enumerate(inst.cap_plus_which):
        ampl.set["MAXPlusWhich"][i+1] = vi
    ampl.param["P"] = inst.P
    ampl.param["S"] = inst.S
    ampl.param["L"] = inst.L
    ampl.param["E"] = inst.E
    ampl.param["Pr"] = pd.DataFrame(data = inst.Pr,
                                    index = range(1, len(inst.P)+1),
                                    columns = range(1, inst.nT+1))
    ampl.param["MAX"] = inst.MAX
    ampl.param["MAXPlus"] = inst.MAXPlus
    
    return ampl

Function to display solution characteristics.

In [9]:
import matplotlib.pyplot as plt 
  
def PresentSolution(ampl):
    for obj in ampl.get_objectives():
        print("{:20} = {}".format(obj[0], obj[1].get_values().to_pandas().T))
    # Preferences
    prefs = ampl.get_objective("PrefViolRanked").get_values().to_pandas()
    plot = prefs.plot(title="Ranked normalized preference violations", xlabel="Seniority")
    prefViolAve = prefs.sum().sum()/len(prefs.index)
    plot.hlines(prefViolAve, 0, len(prefs.index), label="Average", linestyle='--', color='pink')
    plt.legend(('PrefViolRanked', 'Average'))
    plt.show()
    print("AVERAGE NORMALIZED PREFERENCE VIOLATION = {:.3}\n".format(prefViolAve))
    # Solution as a heat map
    x = ampl.get_variable("x").get_values().to_pandas().unstack().T
    plt.imshow( x )
    plt.title( "Schedule" )
    plt.xlabel( "Trainees" )
    plt.ylabel( "Classes" )
    plt.show()
    # Free capacity
    cap = ampl.get_data("Capacity.slack").to_pandas().unstack().T
    cap.columns = inst.cap_kind
    cap.index = np.arange(1, inst.nT+1)
    plot = cap.plot(title="Free capacity", xlabel="Class slot", kind="bar", stacked=True)
    plt.show()
    # Free group capacity
    cap = ampl.get_data("Capacity_Meta.slack").to_pandas().unstack().T
    cap.columns = inst.cap_plus_kind
    cap.index = np.arange(1, inst.nT+1)
    plot = cap.plot(title="Free group capacity", xlabel="Class slot", kind="bar", stacked=True)
    plt.show()
    print("AVERAGE SESSION LOAD IMBALANCE = {:.3}\n".format(
        ampl.get_value(
            "sum {t in T} abs( sum {i in I: t in V[i]} x[i, t] - sum {i in I, t1 in V[i]} x[i, t1] / card(T) ) / card(T)")))
    # Session language
    print("Session language")
    print(ampl.get_variable("u").get_values().to_pandas().T)
    assert 1==min(ampl.get_data("Language_1").to_pandas().T)      # Minimal value of the logical constraints (1 when all true)
    assert 1==min(ampl.get_data("Language_2").to_pandas().T)

## Experiment

We solve the instance with several parameter settings.

### Solve with ranked preferences

To solve with properly ranked preferences as in [1], we submit the model as-is. The solver uses the `.objpriority` suffixes for ranking.

In [ ]:
ampl = MakeAMPLInstance(inst)

ampl.solve(solver="gurobi", gurobi_options="outlev=1 iis=1 multiobj=1 tech:timing=1", verbose=True)

Gurobi 11.0.1: Set parameter LogToConsole to value 1
  tech:outlev = 1
  alg:iisfind = 1
  obj:multi = 1
  tech:timing = 1
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 630 rows, 2636 columns and 8919 nonzeros
Model fingerprint: 0x04ce5b1f
Model has 240 general constraints
Variable types: 1 continuous, 2635 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 203 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applyin

In [ ]:
print("Solve result = {}, time: {:.3}s\n--------------------------------".format(ampl.solve_result, ampl.get_value("Initial.time")))
PresentSolution(ampl)

### Solve with averaged preferences

To reduce the average preference violation, we could sacrifice strict seniority ranking
and minimize the total violation in one go. For this we assign equal
`.objpriority` suffixes for the `PrefViolRanked` objectives. In practice, the sum can
be weighted to keep seniority aspirations, for example by the `.objweight` suffixes.

In [ ]:
ampl.eval("let {s in SenLevels} PrefViolRanked[s].objpriority := 3;")

ampl.solve(solver="gurobi", gurobi_options="outlev=1 iis=1 multiobj=1 tech:timing=1", verbose=True)

In [ ]:
print("Solve result = {}, time: {:.3}s\n--------------------------------".format(ampl.solve_result, ampl.get_value("Initial.time")))
PresentSolution(ampl)

### Solve minimizing session load imbalance

Finally, similar to the last model extension in [1], we post-process the solution
to minimize session load imbalance. We only tackle the imbalance in overall class load.
For that we add yet another objective function.

In [ ]:
ampl.eval(
    """
    # Minimize overall session load imbalance
    minimize LoadImbalance:
      sum {t in T} abs( sum {i in I: t in V[i]} x[i, t] - sum {i in I, t1 in V[i]} x[i, t1] / card(T) )
      suffix objpriority 0;
    """
)

In [ ]:
ampl.solve(solver="gurobi", gurobi_options="outlev=1 iis=1 multiobj=1 tech:timing=1", verbose=True)

In [ ]:
print("Solve result = {}, time: {:.3}s\n--------------------------------".format(ampl.solve_result, ampl.get_value("Initial.time")))
PresentSolution(ampl)